In [21]:
import pandas as pd
import fastparquet


In [22]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(2940, 16)

In [23]:
len(df[df.tiling_compatible == True]) 

75

In [24]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format   1203
1       failed_to_extract    647
2          cant_open_file    551
3         group_structure    191
4  tile_generation_failed    179
5        no_xy_dimensions     79
6                    None     75
7  cant_extract_variables     15


In [35]:
df[df.incompatible_reason == 'cant_open_file'].error_message.unique()


array(["Error opening https://asdc.larc.nasa.gov/data/CERES/SSF1deg-Day/Aqua-MODIS_Edition4A/2018/12/CER_SSF1deg-Day_Aqua-MODIS_Edition4A_404405.201812.hdf: did not find a match in any of xarray's currently installed IO backends ['h5netcdf', 'scipy', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:\nhttps://docs.xarray.dev/en/stable/getting-started-guide/installing.html\nhttps://docs.xarray.dev/en/stable/user-guide/io.html",
       "Error opening https://asdc.larc.nasa.gov/data/CERES/SSF1deg-Day/Terra-MODIS_Edition4A/2016/10/CER_SSF1deg-Day_Terra-MODIS_Edition4A_403405.201610.hdf: did not find a match in any of xarray's currently installed IO backends ['h5netcdf', 'scipy', 'rasterio', 'zarr']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:\nhttps://docs.xarray.dev/en/stable/getting-started-gu

In [38]:
list(df[df.incompatible_reason == 'cant_open_file'].collection_concept_id)

['C1000000600-LARC_ASDC',
 'C1000000601-LARC_ASDC',
 'C1000000602-LARC_ASDC',
 'C1000000680-LARC_ASDC',
 'C1000000681-LARC_ASDC',
 'C1000000701-LARC_ASDC',
 'C1000000702-LARC_ASDC',
 'C1000000703-LARC_ASDC',
 'C1000000704-LARC_ASDC',
 'C1000000705-LARC_ASDC',
 'C1000000720-LARC_ASDC',
 'C1000000721-LARC_ASDC',
 'C1000000722-LARC_ASDC',
 'C1000000723-LARC_ASDC',
 'C1000000724-LARC_ASDC',
 'C1000000725-LARC_ASDC',
 'C1000000726-LARC_ASDC',
 'C1000000727-LARC_ASDC',
 'C1000000728-LARC_ASDC',
 'C1000000729-LARC_ASDC',
 'C1000000730-LARC_ASDC',
 'C1000000742-LARC_ASDC',
 'C1000000746-LARC_ASDC',
 'C1000000747-LARC_ASDC',
 'C1000000762-LARC_ASDC',
 'C1000000763-LARC_ASDC',
 'C1000000764-LARC_ASDC',
 'C1000000765-LARC_ASDC',
 'C1000000766-LARC_ASDC',
 'C1000000771-LARC_ASDC',
 'C1000000772-LARC_ASDC',
 'C1000000773-LARC_ASDC',
 'C1000000774-LARC_ASDC',
 'C1000000775-LARC_ASDC',
 'C1000000776-LARC_ASDC',
 'C1000000777-LARC_ASDC',
 'C1000000778-LARC_ASDC',
 'C1000000779-LARC_ASDC',
 'C100000078

In [33]:
# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and (x.size > 0 and x[0] != None))
)]
subset_df.data_variables

27                                               [SWIR1]
29                                               [TIRS2]
34     [latitude, longitude, scan_datetime, year, mon...
37     [aerosol_optical_thickness_550_land, aerosol_o...
43     [Aerosol_Optical_Thickness_550_Expected_Uncert...
                             ...                        
623    [Delta_Time, SZA, VZA, SAz, VAz, Longitude, La...
625                                             [weight]
640    [obs_id, fov_obs_id, obs_time_tai93, obs_time_...
643    [EFLUXICE, EFLUXWTR, FRSEAICE, HFLUXICE, HFLUX...
646                    [OriginalInputFiles, XmlMetadata]
Name: data_variables, Length: 160, dtype: object

In [47]:
all_unique_vars = set().union(*subset_df.data_variables)

keywords = ['time', 'latitude', 'longitude', 'num_obs', 'foo', 'crs']
filtered_list = [item for item in all_unique_vars if not any(keyword in item.lower() for keyword in keywords)]
print(len(filtered_list))

509


In [36]:
with open('titiler_cmr_compatibility/known_vars.txt', 'w') as f:
    for item in filtered_list:
        f.write(f"{item}\n")

In [11]:
# !aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
#     s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
